# Estimate parameters

In [2]:
# Import libraries
import statistics as stat
import pandas as pd
from scipy import optimize,arange
import numpy as np
from IPython.display import Image

In [3]:
# Load data
df = pd.read_csv("data/V3_timeseries__postrev_I1Y_19-Oct-2018.csv")
#df = pd.read_csv(r"C:\Users\Martin Nørgaard\Documents\GitHub\bachelor\data\V3_timeseries__postrev_I1Y_19-Oct-2018.csv")
df_focus = df[['year','ProfitK','ProfitY','RF', 'growthpop','priceinvt','PD','growthTFP','ik','EmpPop']]
df_focus = df_focus.set_index('year')

In [4]:
df_focus.head()

,ProfitK,ProfitY,RF,growthpop,priceinvt,PD,growthTFP,ik,EmpPop
year,,,,,,,,,
1960,14.234645,29.143753,2.101667,1.646313,-1.321805,28.767187,0.647897,7.456101,56.108616
1961,14.418177,29.418518,1.855833,1.292520,-1.373270,33.813931,1.891988,7.261791,55.354179
1962,15.298585,30.143707,1.725833,1.157498,-1.162756,29.106068,3.314460,7.622490,55.513981
1963,16.091473,31.001640,2.081667,1.866188,-1.561601,30.914055,2.858746,7.922162,55.351719
1964,16.331043,31.335564,2.345833,1.675677,-0.907530,32.665569,2.761517,8.154409,55.670570


## First subsample

### Population growth, investment-specific progress, TFP growth and labour supply

Farhi and Gourio (2018) use measured equivalents for the following model variables:

In [16]:
# Population growth (moment) = population growth (parameter)
g_L = stat.mean(df_focus.loc[1984:2000,"growthpop"])/100
print("g_L = " + str(g_L))

# Investment price growth (moment) = investment-specific progress (parameter)
g_Q = stat.mean(df_focus.loc[1984:2000,"priceinvt"])/100
print("g_Q = " + str(g_Q))

# Employment-population ratio (moment) = labour supply (parameter)
N_bar = stat.mean(df_focus.loc[1984:2000,"EmpPop"])/100
print("N_bar = " + str(N_bar))

# TFP growth (moment) = TFP growth (parameter)
g_Z = stat.mean(df_focus.loc[1984:2000,"growthTFP"])/100
print("g_Z = " + str(g_Z))

g_L = 0.01170647427647059
g_Q = -0.01769225337647059
N_bar = 0.6234416447058824
g_Z = 0.011003753694117647


### Further moment conditions

GDP trend growth may be derived from:

\begin{align}
1 + g_T = (1 + g_L )(1+ g_Z)^{\frac{1}{1-\alpha}}(1+ g_Q)^{\frac{1}{1-\alpha}} \tag{eq. 11} 
\end{align}

The following equations in Farhi and Gourio (2018) are the applied moment conditions, cf. section IV.B on identification:

\begin{align}
\frac{I}{K} = \frac{\Pi}{K / Q} & \simeq \delta+g_{Q}+g_{T} \tag{eq. 18} \\
\frac{P^{\star}}{D^{\star}} & \simeq \frac{1+g_{T}}{r^{\star}-g_{T}} \tag{eq. 23} \\
s_{K} = \frac{\Pi}{Y} = 1-s_{L} & =\frac{\mu+\alpha-1}{\mu} \tag{eq. 20}\\
MPK = \frac{\Pi}{K} & =\frac{\mu+\alpha-1}{\alpha}\left(r^{\star}+\delta+g_{Q}\right) \tag{eq. 27}
\end{align}

We calculate the measured moments (mean values) - M# in parenthesis indicate momentnumbering from Farhi and Gourio pp. 172-173:

In [17]:
# Gross Profitability (M1)
ProfitK = stat.mean(df_focus.loc[1984:2000,"ProfitK"])/100
print("Profit/K = " + str(ProfitK))

# Gross Capital Share (M2)
s_K = stat.mean(df_focus.loc[1984:2000,"ProfitY"])/100
print("s_K = " + str(s_K))

# Investment-capital ratio (M3)
ik = stat.mean(df_focus.loc[1984:2000,"ik"])/100
print("I/K = " + str(ik))

# Price-Dividend Ratio (M5)
PD = stat.mean(df_focus.loc[1984:2000,"PD"])/100
print("P/D = " + str(PD))

Profit/K = 0.14011972529411765
s_K = 0.29887441294117645
I/K = 0.08102986358823529
P/D = 0.4233587782352941


In [23]:
#Set system of equations
def eq_system(parameters, g_L, g_Q, g_Z, ProfitK, s_K, ik, PD):
    
    alpha = parameters[0]
    g_T = parameters[1]
    delta = parameters[2]
    r = parameters[3]
    mu = parameters[4]
    

    eq_11 = (1+g_L)*((1+g_Z)**(1/(1-alpha)))*((1+g_Q)**(alpha/(1-alpha))) - (1+g_T)
    eq_18 = delta + g_Q + g_T - ik
    eq_23 = (1+g_T)/(r-g_T) - PD
    eq_20 = (mu + alpha - 1)/mu - s_K
    eq_27 = ((mu + alpha - 1)/alpha)*(r + delta + g_Q) - ProfitK

    return [eq_11, eq_18, eq_23, eq_20, eq_27]

> OBS: I Farhi og Gourios MATLAB-fil bruger de følgende ligning

\begin{align}
1 + g_T = (1 + g_L )(1+ g_Z)^{\frac{1}{1-\alpha}}(1+ g_Q)^{\frac{\alpha}{1-\alpha}} \tag{eq. 11} 
\end{align}

In [24]:
moments = (g_L, g_Q, g_Z, ProfitK, s_K, ik, PD)

#From MATLAB:
#alpha = 0.3, g_T = ?, delta = 4.0, r = ?, mu = 1.00

guess1 = [0.2,0.1,0.2,0.4,0.2]
guess2 = [0.3,0.04,0.03,0.06,0.06]
guess3 = [0.2,0.04,0.03,0.05,0.08]

solution1 = optimize.fsolve(eq_system, guess1, args=moments, full_output=0)
solution2 = optimize.fsolve(eq_system, guess2, args=moments, full_output=0)
#solution3 = optimize.fsolve(eq_system, guess3, args=moments, full_output=0)

print("alpha er estimeret til " + str(round(solution1[0],3)))
print("g_T er estimeret til " + str(round(solution1[1],3)))
print("delta er estimeret til " + str(round(solution1[2],3)))
print("r er estimeret til " + str(round(solution1[3],3)))
print("mu er estimeret til " + str(round(solution1[4],3)))
print("")

print("alpha er estimeret til " + str(round(solution2[0],3)))
print("g_T er estimeret til " + str(round(solution2[1],3)))
print("delta er estimeret til " + str(round(solution2[2],3)))
print("r er estimeret til " + str(round(solution2[3],3)))
print("mu er estimeret til " + str(round(solution2[4],3)))
print("")

#print("alpha er estimeret til " + str(round(solution3[0],3)))
#print("g_T er estimeret til " + str(round(solution3[1],3)))
#print("delta er estimeret til " + str(round(solution3[2],3)))
#print("r er estimeret til " + str(round(solution3[3],3)))
#print("mu er estimeret til " + str(round(solution3[4],3)))
print("")

print("mu = " + str(round(solution2[4],3)))
print("delta = " + str(round(solution1[2],3)))
print("alpha = " + str(round(solution2[0],3)))
print("g_L = " + str(round(g_L*100,3)))
print("g_Z = " + str(round(g_Z*100,3)))
print("g_Q = " + str(round(-g_Q*100,3)))
print("N_bar = " + str(round(N_bar,3)))

alpha er estimeret til 0.879
g_T er estimeret til -0.027
delta er estimeret til 0.126
r er estimeret til 2.271
mu er estimeret til 0.173

alpha er estimeret til 0.879
g_T er estimeret til -0.027
delta er estimeret til 0.126
r er estimeret til 2.271
mu er estimeret til 0.173


mu = 0.173
delta = 0.126
alpha = 0.879
g_L = 1.171
g_Z = 1.1
g_Q = 1.769
N_bar = 0.623


![title](images/Estimates.png)

### Risk Free Rate

We may finally deduce the risk premium using data on the risk-free interest rate. This however requires further assumptions.

In [ ]:
#Intertemporal elasticity of substitution
IES = 2

# Rare disaster distribution (Chi = 15 pct.)
e_b = 0.85

# Risk aversion coefficient
theta = 12